
#  Naive Bayes: Teoría y Aplicación Práctica


"""
# 🌾 Clasificador Naive Bayes (Teoría y Práctica)

El **método Naive Bayes** es un clasificador probabilístico basado en el **Teorema de Bayes**,
que calcula la probabilidad de que una observación pertenezca a una clase dada la evidencia de sus características.

## 🧠 Teoría Básica

El teorema de Bayes establece que:

P(C|X) = [ P(X|C) * P(C) ] / P(X)


Donde:

- **P(C | X)** → Probabilidad de que el mensaje pertenezca a una clase *C* (por ejemplo, “spam”) después de ver el texto *X*.  
- **P(X | C)** → Probabilidad de ver ese texto *X* si ya sabemos que es de la clase *C*.  
- **P(C)** → Probabilidad de que un mensaje sea de la clase *C* antes de leerlo.  
- **P(X)** → Probabilidad total de ver el texto *X* (es igual para todas las clases).


El término “**Naive**” (ingenuo) se debe a que asume que **todas las características son independientes entre sí**,
lo cual rara vez es cierto en la práctica, pero el modelo sigue funcionando sorprendentemente bien.

## ⚙️ Variantes Comunes
- **GaussianNB**: para variables continuas (valores numéricos).
- **MultinomialNB**: para conteos o frecuencias (por ejemplo, texto).
- **BernoulliNB**: para variables binarias (palabras presentes o ausentes).

## 🎛️ Hiperparámetros importantes
- **alpha**: suavizado de Laplace (evita que probabilidades con cero frecuencia anulen el modelo).
  - Valores altos → más suavizado, menor influencia de palabras raras.
  - Valores bajos → menos suavizado, más sensible a palabras poco frecuentes.
- **fit_prior**: si se usa o no la probabilidad previa de las clases.

En este ejemplo veremos cómo afectan estos hiperparámetros al rendimiento del modelo.
"""


In [7]:
# ================================================
# 🧪 Ejemplo Práctico: Clasificación de Mensajes
# ================================================

#!pip install scikit-learn matplotlib

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import matplotlib.pyplot as plt

# ===========================================================
# 📚 Dataset
# ===========================================================

mensajes = [
    # SPAM
    "gana dinero rápido con este método",
    "oferta exclusiva para ti",
    "compra ahora y recibe un descuento",
    "felicitaciones has sido seleccionado",
    "obtén un préstamo sin intereses",
    "recibe dinero fácil y rápido",
    "tu cuenta ha sido elegida para un premio",
    "ganaste un viaje a la playa totalmente gratis",
    "oferta limitada solo hoy",
    "aprovecha esta promoción especial",

    # NO SPAM
    "reunión del departamento mañana",
    "recordatorio de cita médica",
    "tus notas del examen están listas",
    "la tarea debe entregarse hoy",
    "nos vemos en la escuela",
    "gracias por tu ayuda con el informe",
    "revisemos el trabajo antes de entregarlo",
    "hoy toca clase de matemáticas",
    "feliz cumpleaños te deseo lo mejor",
    "confirma tu asistencia a la reunión"
]

etiquetas = [
    "spam", "spam", "spam", "spam", "spam",
    "spam", "spam", "spam", "spam", "spam",
    "no_spam", "no_spam", "no_spam", "no_spam", "no_spam",
    "no_spam", "no_spam", "no_spam", "no_spam", "no_spam"
]



### ✉️ Vectorización de texto

Los algoritmos de aprendizaje automático no pueden trabajar directamente con palabras,
ya que necesitan números para hacer cálculos.  
Por eso, usamos un **vectorizador**, que convierte cada mensaje en un conjunto de números.

- **`CountVectorizer()`** toma todos los mensajes y construye un “diccionario” con las palabras que aparecen.  
- Luego, **`fit_transform(mensajes)`** crea una matriz donde:
  - Cada **fila** representa un mensaje.
  - Cada **columna** representa una palabra.
  - El valor en cada posición indica **cuántas veces aparece esa palabra** en el mensaje.

Así, el texto se transforma en una forma numérica que el modelo Naive Bayes puede entender.


In [8]:
# Vectorización de texto
vectorizador = CountVectorizer()
X = vectorizador.fit_transform(mensajes)


In [9]:
# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, etiquetas, test_size=0.25, random_state=42)

### ⚙️ Explorando el efecto de los hiperparámetros

En esta parte probamos **distintas configuraciones del modelo Naive Bayes** para ver cómo cambian los resultados.

Los hiperparámetros que estamos ajustando son:

- **`alpha`** → controla el *suavizado de Laplace*.  
  Sirve para evitar que una palabra con frecuencia cero arruine los cálculos del modelo.  
  - Valores pequeños → el modelo se ajusta más a los datos (menos general).  
  - Valores grandes → el modelo se suaviza más (más general).

- **`fit_prior`** → indica si el modelo debe usar o no las probabilidades previas de las clases.  
  - `True`: considera cuántos ejemplos hay de cada clase en el conjunto de entrenamiento.  
  - `False`: trata ambas clases como si fueran igual de probables.

El bucle `for` prueba varias combinaciones de estos dos parámetros, entrena un modelo para cada una y guarda su **precisión** (exactitud) en una tabla.  
Así podemos comparar qué valores dan el mejor rendimiento.


In [10]:

# ===========================================================
# 🔧 Explorando el efecto de los hiperparámetros
# ===========================================================

resultados = []

# Probaremos diferentes valores de alpha y fit_prior
for alpha in [0.01, 0.1, 0.5, 1.0, 2.0, 5.0]:
    for prior in [True, False]:
        modelo = MultinomialNB(alpha=alpha, fit_prior=prior)
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        resultados.append({
            "alpha": alpha,
            "fit_prior": prior,
            "accuracy": acc
        })

df_resultados = pd.DataFrame(resultados)
print("Resultados con diferentes hiperparámetros:\n")
print(df_resultados)

Resultados con diferentes hiperparámetros:

    alpha  fit_prior  accuracy
0    0.01       True       0.8
1    0.01      False       0.8
2    0.10       True       0.8
3    0.10      False       0.8
4    0.50       True       0.8
5    0.50      False       0.8
6    1.00       True       0.8
7    1.00      False       0.8
8    2.00       True       0.8
9    2.00      False       0.8
10   5.00       True       0.8
11   5.00      False       0.8


In [13]:

# ===========================================================
# 🏁 Entrenamos el mejor modelo y mostramos detalles
# ===========================================================
mejor = df_resultados.sort_values("accuracy", ascending=False).iloc[0]
modelo_final = MultinomialNB(alpha=mejor.alpha, fit_prior=mejor.fit_prior)
modelo_final.fit(X_train, y_train)
y_pred_final = modelo_final.predict(X_test)

print("\n🔹 Mejor configuración encontrada:")
print(mejor)
print("\n🔹 Reporte de clasificación:")
print(classification_report(y_test, y_pred_final))

# Ejemplo de predicción nueva
nuevo_mensaje = ["obtén un descuento especial hoy"]
nuevo_vec = vectorizador.transform(nuevo_mensaje)
print("🔹 Predicción para mensaje nuevo:", modelo_final.predict(nuevo_vec)[0])



🔹 Mejor configuración encontrada:
alpha        0.01
fit_prior    True
accuracy      0.8
Name: 0, dtype: object

🔹 Reporte de clasificación:
              precision    recall  f1-score   support

     no_spam       0.67      1.00      0.80         2
        spam       1.00      0.67      0.80         3

    accuracy                           0.80         5
   macro avg       0.83      0.83      0.80         5
weighted avg       0.87      0.80      0.80         5

🔹 Predicción para mensaje nuevo: spam


## 🧮 Ejemplo 2: Clasificación con Gaussian Naive Bayes

En este ejemplo usaremos el modelo **GaussianNB**, que se aplica a variables **numéricas continuas**.  
A diferencia de *MultinomialNB* (que trabaja con texto y conteos de palabras),  
este modelo asume que los datos siguen una **distribución normal (gaussiana)** dentro de cada clase.

Vamos a crear un conjunto pequeño con dos características:
- **Altura (cm)**
- **Peso (kg)**

El objetivo será clasificar a cada persona en uno de dos grupos (`A` o `B`) según sus medidas.

---

### 🌾 ¿Qué hace el método Naive Bayes aquí?

El clasificador **Naive Bayes** usa el **Teorema de Bayes** para calcular la probabilidad de que un caso pertenezca a una clase.

\[
P(C|X) = \frac{P(X|C) \times P(C)}{P(X)}
\]

Donde:
- **P(C|X)** → probabilidad *a posteriori*: qué tan probable es que el ejemplo pertenezca a la clase *C* después de ver los datos *X*.  
- **P(X|C)** → probabilidad de observar esos valores si ya sabemos que es de la clase *C*.  
- **P(C)** → probabilidad *a priori* de la clase *C* (qué tan frecuente es).  
- **P(X)** → probabilidad total de observar los valores de *X* (igual para todas las clases).

---

### 🧠 ¿Por qué “Naive” (ingenuo)?

Porque el modelo **supone que las variables son independientes entre sí**.  
En nuestro ejemplo, eso significa asumir que la *altura* y el *peso* no dependen una de la otra,  
aunque en la realidad están algo relacionadas.  
Aun así, esta simplificación suele dar **muy buenos resultados**.

---

### ⚙️ ¿Qué hace GaussianNB exactamente?

1. Calcula la **media (μ)** y la **desviación estándar (σ)** de cada variable dentro de cada grupo.  
2. Usa la **fórmula de la distribución normal** para estimar la probabilidad de ver un valor dado.  
3. Combina esas probabilidades para obtener la probabilidad total de que un individuo pertenezca a cada clase.  
4. Elige la clase con la probabilidad *a posteriori* más alta.

---

### 📈 Intuición simple

- Si alguien tiene **altura y peso cercanos** a los promedios del grupo B, el modelo lo clasificará como **B**.  
- Si sus valores están más cerca de los del grupo A, lo asignará a **A**.  
- Todo esto se hace usando **probabilidades**, no comparaciones directas.

---

En resumen, **Gaussian Naive Bayes** usa el teorema de Bayes con la suposición de que los datos numéricos de cada clase siguen una curva normal.  
Es rápido, sencillo y funciona muy bien cuando los datos se distribuyen de forma más o menos “redondeada” alrededor de sus promedios.


In [14]:
# ===========================================================
# 📊 Ejemplo con datos numéricos — GaussianNB
# ===========================================================

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd



In [15]:
# Dataset pequeño y equilibrado
datos = pd.DataFrame({
    "altura": [160, 165, 170, 155, 180, 175, 185, 150, 172, 168],
    "peso":   [60, 65, 68, 50, 80, 75, 85, 48, 70, 66],
    "grupo":  ["A", "A", "A", "A", "B", "B", "B", "A", "B", "B"]
})

print("📋 Dataset inicial:")
print(datos)

# Variables (X) y etiquetas (y)
X = datos[["altura", "peso"]]
y = datos["grupo"]

📋 Dataset inicial:
   altura  peso grupo
0     160    60     A
1     165    65     A
2     170    68     A
3     155    50     A
4     180    80     B
5     175    75     B
6     185    85     B
7     150    48     A
8     172    70     B
9     168    66     B


In [16]:
# Dividimos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
# ===========================================================
# 🧠 Entrenamiento con GaussianNB
# ===========================================================
modelo = GaussianNB()
modelo.fit(X_train, y_train)

# Predicciones
y_pred = modelo.predict(X_test)

# Resultados
print("\n🔹 Precisión del modelo:", accuracy_score(y_test, y_pred))
print("\n🔹 Reporte de clasificación:")
print(classification_report(y_test, y_pred))


🔹 Precisión del modelo: 1.0

🔹 Reporte de clasificación:
              precision    recall  f1-score   support

           A       1.00      1.00      1.00         1
           B       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3



In [18]:
# Ejemplo nuevo
nuevo = [[178, 77]]  # Altura 178 cm, peso 77 kg
prediccion = modelo.predict(nuevo)[0]
print(f"\n🔹 Predicción para altura=178, peso=77 → Grupo {prediccion}")


🔹 Predicción para altura=178, peso=77 → Grupo B


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
